## Projet ANLO : Analyse de données aériennes
Anne DE JONG & Loris MERCIER
2022

Questions :
- Filtrer df --> Jointure depuis dictionnaire pour grouper... temps ? --> Créer nouvelle df
- Nettoyage --> temps raisonnable pour cela ? --> Faire une fois avant
- A value is try to be set... --> df.Copy


In [ ]:
import pandas as pd
import numpy as np
import csv
import math


#Plot chart
import plotly.express as px

#Widget
import ipywidgets as widgets
from IPython.display import display

#Map interactive
import folium as f
from folium.plugins import MarkerCluster

#Map pour connection
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap

#Calcul great circle
import pyproj


### Importation des données <a class="anchor" id="chapter0"></a>

In [ ]:
chemin="../data/"

#df_aeroports = pd.read_csv(chemin + "aeroportsANLO.csv")
#df_zones = pd.read_csv(chemin + "zonesANLO.csv")
df_vols = pd.read_csv(chemin + "routesANLO.csv")
#df_compagnies = pd.read_csv(chemin + "compagniesANLO.csv")

In [ ]:
dict_compagnies = {}

with open(chemin+'compagniesANLO.csv', mode='r') as inp:
    reader = csv.reader(inp)
    next(reader, None)
    headers = ['Name','Country']
    dict_compagnies = {rows[2]:dict(zip(headers,[rows[0],rows[3]])) for rows in reader}

dict_compagnies

In [ ]:
dict_aeroports = {}

with open(chemin+'aeroportsANLO.csv', mode='r') as inp:
    reader = csv.reader(inp)
    headers = next(reader, None)
    dict_aeroports = {rows[1]:dict(zip(headers,rows[0:8])) for rows in reader}

dict_aeroports

In [ ]:
dict_pays = {}

with open(chemin+'countries.csv', mode='r') as inp:
    reader = csv.reader(inp)
    headers = next(reader, None)
    dict_pays = {rows[1]:rows[0] for rows in reader}

dict_pays

Fonction de calcul de distance

In [ ]:
def coordDepuisAeroport(iata : str):
    '''
    Fonction : Recherche les coordonnées GPS d'un aéroport
    Retour : lat : float ,lon : float --> Latitude & longitude de l'aéroport
    '''
    lat = float(dict_aeroports[iata]['Lat'])
    lon = float(dict_aeroports[iata]['Lon'])
    return lat,lon

In [ ]:
def DistGrandCercle(lat1 : float, lon1 : float, lat2 : float, lon2 : float):
    #Conversion en radian
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    #Delta des coordonnées
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Rayon terrestre (en km)
    r = 6367.0

    #Formule d'Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return 2 * r * math.asin(math.sqrt(a))


def DistGrandCercleICAO(icao1 : str, icao2 : str):
    #Récupération des coordonnées des aéroports
    latDep, lonDep = coordDepuisAeroport(icao1)
    latArr, lonArr = coordDepuisAeroport(icao2)

    #On calcule la distance les séparant
    return DistGrandCercle(latDep,lonDep,latArr,lonArr)

#Lyon Latitude : 45.7603831. Longitude : 4.849664
#Montreal Latitude :	45.5016889  Longitude -73.567256
DistGrandCercle(45.7603831,4.849664,45.5016889,-73.567256)


In [ ]:
dict_route = {}

g = df_vols.groupby(['Depart','Arrivee'])
taille_groupe = g.size().items()

for liaison, nbrVols in taille_groupe:
    #Si la liaison est déjà dans le dict, on incrémente le nbr de vols
    if(frozenset(liaison) in dict_route):
        dict_route[frozenset(liaison)]['Vols'] += nbrVols
    #Sinon, on crée une nouvelle liaison
    else:
        dict_route[frozenset(liaison)] = {'Dist' : DistGrandCercleICAO(liaison[0],liaison[1]), 'Vols' : nbrVols}

#Exemple :
dict_route[frozenset(('LYS','CDG'))]

Fonction pour filtrer les dictionnaires

In [ ]:
#bidict
def filtreDictAeroports(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des aéroports
    '''
    Listtmp = []
    for (key, value) in dict_aeroports.items():
        if value[colonne] == valeur:
            Listtmp.append(key)
    return Listtmp

def filtreDictCompagnies(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des compagnies
    '''
    Listtmp = []
    for (key, value) in dict_compagnies .items():
        if value[colonne] == valeur:
            Listtmp.append(key)
    return Listtmp

### Analyse géographique <a class="anchor" id="chapter1"></a>

Liste aéroport

In [ ]:
m = f.Map(
    location = [0, 0],
    zoom_start = 2
)

cluster = MarkerCluster(name='Aeroport').add_to(m)
f.LayerControl().add_to(m)


for liste in dict_aeroports.values():
    f.Marker(
        location = [liste['Lat'], liste['Lon']], #Position sur la carte
        popup= "<b>Nom : </b>" + liste['Name'] + """<br />
        <b>Code :</b> """ + liste['Iata'] + """<br />
        <b>Coordonnées :</b> (""" + str(liste['Lat']) + ";" + str(liste['Lon']) + """)<br />
        <b>City : </b>""" + str(liste['City']) + """<br />
        <b>Pays : </b>""" + liste['Country'],
        tooltip=liste['City'], #hoover over 
        icon= f.Icon(icon="plane")
    ).add_to(cluster)



m

Map connection (A supprimer)

In [ ]:
#Mise en forme des DF pour l'afficha de la map connection
def liste_aeroport():
    liste_aeroport_unique =[]
    for depart in df_vols['Depart'].unique():
        if depart in dict_aeroports:
            liste_aeroport_unique.append(dict_aeroports[depart]['Name']) #Nom aéroport

    return sorted(liste_aeroport_unique)

def dfvols_avec_coordonnees(depart,compagnie):
    if(compagnie != ""):
        df_tmp = pd.merge(df_vols[df_vols["Depart"] == depart][["Depart","Arrivee","Code_ICAO"]], #Vol filtré avec depart
                            df_compagnies[df_compagnies["Name"] == compagnie][["Code_ICAO","Name"]], #Vol filtré avec compagnie
                            how="inner", on="Code_ICAO")
    else:
        df_tmp = df_vols[df_vols["Depart"] == depart][["Depart","Arrivee"]]

    return pd.merge(df_tmp,
                df_aeroports[['Iata','Lat','Lon']].rename(columns={'Iata' : "Depart", "Lat" : "latDep", "Lon" : "lonDep"}), 
                how="left", on="Depart")\
            .merge(df_aeroports[['Iata','Lat','Lon']].rename(columns={'Iata' : "Arrivee","Lat" : "latArr", "Lon" : "lonArr"}), 
            how="left", on="Arrivee")

#Fonction de calculs
def Sqr(a):
    return a*a

def Dist(x1,y1,x2,y2):
    return math.sqrt((y2-y1)**2+(x2-x1)**2)

In [ ]:
"""
output = widgets.Output()
plt.rcParams["figure.figsize"]=15,12


def tracer_carte(zone):
    try:
        m=Basemap(lat_0=0, lon_0=0, 
             llcrnrlon=df_zones[df_zones['Zone'] == zone].iat[0,1], #lonInf
            llcrnrlat=str(df_zones[df_zones['Zone'] == zone].iat[0,2]), #latInf
            urcrnrlon=str(df_zones[df_zones['Zone'] == zone].iat[0,3]), #lonSup
            urcrnrlat=str(df_zones[df_zones['Zone'] == zone].iat[0,0]), #latSup
            projection='merc')
        m.drawmapboundary(fill_color='white', linewidth=0)
        m.fillcontinents(color='grey', alpha=0.7)
        m.drawcoastlines(linewidth=0.4, color="white")
        m.drawcountries(linewidth=0.4, color="white")
    except:
        pass
    return m

def tracer_vol_depuis(depart,compagnie,m):
    #conversion en code ICAO
    depart = df_aeroports[df_aeroports['Name'] == depart].iat[0,1]

    #Jointure entre df pour obtenir une table (Depart,Arrivee, LatDepart, LonDepart, LatArrivee, LonArrivee)
    df_tmp = dfvols_avec_coordonnees(depart,compagnie)   
    for _, ligne in df_tmp.iterrows():
        if(Dist(ligne.lonDep, ligne.latDep, ligne.lonArr, ligne.latArr) > 0.5):     
            m.drawgreatcircle(ligne.lonDep, ligne.latDep, ligne.lonArr, ligne.latArr,linewidth=1, color='#FF4590');
            #plt.annotate("He", xy=m(ligne.lonArr,ligne.latArr,), verticalalignment='center')
            x,y = m(ligne.lonArr, ligne.latArr)
            m.plot(x, y, 'o', color='b')
    
    #Paramètre de la carte
    #print(depart)
    if(compagnie == ""):
        titre = "Connection map à partir de "+ Combdepart.value
    else:
        titre = "Connection map à partir de " + Combdepart.value + " avec " + CombCompagnie.value
    
    print("Nombre de vols : " + str(len(df_tmp)))
    plt.title(titre)
    plt.show()

@output.capture()
def afficheVolDepuis(obj):
    output.clear_output()
    if(Combzone.value.strip() != "" and Combdepart.value.strip() != ""):
        m = tracer_carte(Combzone.value)
        tracer_vol_depuis(Combdepart.value,CombCompagnie.value,m)   
    else :
        print("Selectionner un zoom et un aéroport de départ")


Combzone = widgets.Combobox(
    value='monde',
    placeholder='Choisir une zone',
    options= df_zones['Zone'].to_list(),
    description='Zoom* :',
    ensure_option=True,
    disabled=False
)

Combdepart = widgets.Combobox(
    #value='Lyon Saint Exupery Airport',
    placeholder='Choisir aéroport',
    options= liste_aeroport(),#list(df_volsSD.Depart.unique()),
    description='Aéroport* :',
    ensure_option=True,
    disabled=False,
    layout = widgets.Layout(padding="0px 0px 0px 0px")
)

CombCompagnie = widgets.Combobox(
    placeholder='optionnelle',
    options= df_compagnies.Name.values.tolist(),#list(df_volsSD.Depart.unique()),
    description='Compagnie :',
    ensure_option=True,
    disabled=False,
    layout = widgets.Layout(padding="0px 0px 0px 0px")
)

button = widgets.Button(
    description = 'Afficher',
    button_style = 'info', # '', success', 'info', 'warning' et 'danger'
    layout=widgets.Layout(margin ="3px 0px 0px 20px")
)



button.on_click(afficheVolDepuis)
w = widgets.VBox([widgets.HBox([Combzone,Combdepart]),CombCompagnie,button])
display(w,output)
"""

Map connection : Par aéroports

In [ ]:
def vectGrandCercle(latDep : float, lonDep : float, latArr : float, lonArr : float):
    '''
    Fonction : Calcul un semble de coordonnées permettant de tracer un grand cercle terrestre
    Return : 2 listes de coordonnées
    '''
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(lonDep, latDep, lonArr, latArr)

    # Coordonnée avec des segments <= 100 km
    lonlats = g.npts(lonDep, latDep, lonArr, latArr,
                    1 + int(dist / 100000))

    #Mise en forme
    v1 = []
    v1.append([latDep,lonDep])
    v2 = []
    _lon = lonDep
    _lat = latDep
    horscadre = False
    for lon, lat in lonlats:
        if(not(horscadre) and Dist(_lat,_lon,lat,lon) < 4):        
            v1.append([lat,lon])
        else:
            horscadre = True
            v2.append([lat,lon])
        _lon = lon
        _lat = lat
    
    if(horscadre):
        v2.append([latArr,lonArr])
    else:
        v1.append([latArr,lonArr])

    return v1,v2

In [ ]:
def map_connection(depart,compagnie,m):
    Line_monde = f.FeatureGroup(name = "International")
    Line_continent = f.FeatureGroup(name = "Continental")
    Line_pays = f.FeatureGroup(name = "National")   

    
    df_tmp = df_vols[df_vols['Depart'] == depart]

    for _, ligne in df_tmp.iterrows():
        #Certains aéroports n'ont pas de correspondance dans le dict. Pour éviter l'erreur -> try puis pass
        try:
            latDep, lonDep = coordDepuisAeroport(ligne.Depart)
            latArr, lonArr = coordDepuisAeroport(ligne.Arrivee)

            #Calcul des chemins à afficher
            v2 = []
            v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)

            #Test vol national ou international
            if(dict_aeroports[ligne.Depart]['Country'] == dict_aeroports[ligne.Arrivee]['Country']):
                Line_group = Line_pays
                col_ligne = "#80F0A3"
                col_marker = "#008329"
            elif(dict_pays[dict_aeroports[ligne.Depart]['Country']] == dict_pays[dict_aeroports[ligne.Arrivee]['Country']] ):
                Line_group = Line_continent
                col_ligne = "#F6FF28"
                col_marker = "#FE7700"
            else:
                Line_group = Line_monde
                col_ligne = "#563DFF"
                col_marker = "#0004A8"


            #-------------------
            f.PolyLine(locations=v1,weight=1, color=col_ligne).add_to(Line_group)
            if(len(v2) != 0):
                f.PolyLine(locations=v2,weight=1, color=col_ligne).add_to(Line_group)
            
            f.CircleMarker(location = [latArr, lonArr],
                            radius = 1, 
                            color = col_marker, 
                            tooltip=dict_aeroports[ligne.Arrivee]['City'],
                            popup="<b>Nom : </b>" + dict_aeroports[ligne.Arrivee]['Name'] + """<br />
                                    <b>Code :</b> """ + dict_aeroports[ligne.Arrivee]['Iata'] + """<br />
                                    <b>Coordonnées :</b> (""" + str(dict_aeroports[ligne.Arrivee]['Lat']) + ";" + str(dict_aeroports[ligne.Arrivee]['Lon']) + """)<br />
                                    <b>City : </b>""" + str(dict_aeroports[ligne.Arrivee]['City']) + """<br />
                                    <b>Pays : </b>""" + dict_aeroports[ligne.Arrivee]['Country']).add_to(Line_group)
        except:
            pass

    #Ajout des layers à la carte
    Line_monde.add_to(m)
    Line_continent.add_to(m)
    Line_pays.add_to(m)

#Paramètre de la carte
m = f.Map(
    location = [0, 0],
    zoom_start = 2.3,
    min_zoom  = 2,
    min_lot=-180,
    max_lot=180,
    min_lat=-90,
    max_lat=90,
    max_bounds=True,
)
f.TileLayer('cartodbdark_matter').add_to(m)
map_connection('YUL',"",m)
f.LayerControl().add_to(m)

m

Map connection : Liaisons nationales

In [ ]:
def map_connectionPays(pays,m):
    '''
    Fonction : Dessine les liaisons entre différentes aéroports d'un même pays
    Return : None
    '''
    
    #Code ICAO des aéroports dans le pays en question
    listAeroportPays = filtreDictAeroports('Country',pays)

    #Filtrage des vols en fonction des aéroports possibles
    df_volsFiltre = df_vols[df_vols["Depart"].isin(listAeroportPays) & df_vols["Arrivee"].isin(listAeroportPays)]

    #Nombre de vols par aéroport (Depart + Arrivee)
    Serie_nbrVolsParAeroport =  pd.concat([df_volsFiltre['Depart'],df_volsFiltre['Arrivee']]).value_counts()
    #On groupe par liaison puis on on ajoute la colonne 'count' pour savoir combien de vol par liaison
    df_GroupLiaison = df_volsFiltre.groupby(['Depart','Arrivee']).size().reset_index(name="Count")
    
    Serie_nbrDestinationParAeroport =  df_GroupLiaison['Depart'].value_counts()


    maxLigne = df_GroupLiaison['Count'].max()
    minLigne = df_GroupLiaison['Count'].min()

    print(maxLigne)
    print(minLigne)
    set1=set()

    #Création des Layers
    tabLine = []
    _max = maxLigne/5
    #-0.1 pour ne pas créer un nouveau bloc en cas de division entière
    for i in range(0,int(_max-0.1) + 1):
        tabLine.append(f.FeatureGroup(name = "Entre " + str(int(i*5) + 1) + " et " +
                                              str((int((i+1)*5))) + " liaisons"))

    for _, ligne in df_GroupLiaison.iterrows():
        #Certains aéroports n'ont pas de correspondance dans le dict. Pour éviter l'erreur -> try puis pass
        try:
            latDep, lonDep = coordDepuisAeroport(ligne.Depart)
            latArr, lonArr = coordDepuisAeroport(ligne.Arrivee)

            #Calcul des chemins à afficher
            v2 = []
            v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)
            
            #print(v1)
            #-------------------
            set1.add(ligne.Count/5)

            f.PolyLine(locations=v1,weight=0.2 +ligne.Count/5, color='blue', opacity=0.5,
                        popup="<b>" + dict_aeroports[ligne.Depart]['City'] + " --> " +
                                dict_aeroports[ligne.Arrivee]['City'] +  "</b><br/>" +
                                "<b>Liaisons : </b>" + str(ligne.Count)
                        ).add_to(tabLine[int(ligne.Count/5 - 0.1)])
            
            if(len(v2) != 0):
                f.PolyLine(locations=v2,weight=0.2 +ligne.Count/5, color='blue', opacity=0.5,
                        popup="<b>" + dict_aeroports[ligne.Depart]['City'] + " --> " +
                                dict_aeroports[ligne.Arrivee]['City'] +  "</b><br/>" +
                                "<b>Liaisons : </b>" + str(ligne.Count)
                        ).add_to(tabLine[int(ligne.Count/5 - 0.1)])
            #print(ligne.Depart, " : ",df_GroupDepart[ligne.Depart])
        except:
            print("Erreur entre ", ligne.Depart, " et ", ligne.Arrivee, " -- ", ligne.Count)
            pass
    
    #Ajout des Layers:
    for i in range(0,int(_max-0.1) + 1):
        tabLine[i].add_to(m)

    for ligne in Serie_nbrVolsParAeroport.items():

        if ligne[0] in Serie_nbrDestinationParAeroport.index:
            nbrDest = Serie_nbrDestinationParAeroport[ligne[0]]
        else:
            nbrDest = 0  
        
        Text = f.IFrame("<b>" + dict_aeroports[ligne[0]]['Name'] + """</b><br />
                        <b>City : </b>""" + str(dict_aeroports[ligne[0]]['City']) + """<br />
                        <b>Pays : </b>""" + dict_aeroports[ligne[0]]['Country'] + """<br />
                        <b>Code :</b> """ + dict_aeroports[ligne[0]]['Iata'] + """<br/><br/>
                        <b>Destinations : </b> """ + str(nbrDest))

        #Circle vs CircleMarker
        f.CircleMarker(location= [dict_aeroports[ligne[0]]['Lat'],dict_aeroports[ligne[0]]['Lon']], 
                        radius = 1 + ligne[1]/35, 
                        color='red', 
                        fill_color='red',
                        fill_opacity=1,
                        tooltip=dict_aeroports[ligne[0]]['Name'],                        
                        popup = f.Popup(Text, min_width = 250, max_width = 400)
                        ).add_to(m)       
    
    print(set1)

   
#Paramètre de la carte
m = f.Map(
    location = [46, 2],
    zoom_start = 6,
    min_zoom  = 2,
    min_lot=-200,
    max_lot=200,
    min_lat=-90,
    max_lat=90,
    max_bounds=True,
)
#f.TileLayer('cartodbdark_matter').add_to(m)
map_connectionPays('France',m)
#f.PolyLine(locations=line).add_to(m)
f.LayerControl().add_to(m)

m

Choropleth map

In [ ]:
#Choix de la carte : précis ou pas précis
#world = chemin + "world.geo.json" Pas encore sur GIT
world = chemin + "custom.geo.json"

In [ ]:
pays = "France"
df_tmp = pd.merge(pd.DataFrame.from_dict(dict_aeroports,orient='index')['Country'],
            df_vols[df_vols["Depart"].isin(filtreDictAeroports('Country',pays))],
            left_index=True,
            right_on="Arrivee").groupby('Country').size().reset_index()
df_tmp.drop(df_tmp.loc[df_tmp['Country']==pays].index, inplace=True)

In [ ]:
def ChoroplethNbrLiaisons(pays,m):
    
    df_tmp = pd.merge(pd.DataFrame.from_dict(dict_aeroports,orient='index')['Country'],
            df_vols[df_vols["Depart"].isin(filtreDictAeroports('Country',pays))],
            left_index=True,
            right_on="Arrivee").groupby('Country').size()

    #errores = 'ignore' car certains pays n'ont pas de vol interne. (Cf Singapour)
    df_tmp.drop(pays, inplace=True, errors='ignore')

    f.Choropleth(
        geo_data=world,
        name="choropleth",
        data=df_tmp,
        key_on="feature.properties.name",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=.1,
        legend_name="Nombre de liaisons",
        nan_fill_opacity = 0.1,
        nan_fill_color='white',
        Highlight= True
    ).add_to(m)

pays = 'Netherlands'
m = f.Map(location=[0, 0], 
            zoom_start=2,
            min_zoom  = 2,
            min_lot=-200,
            max_lot=200,
            min_lat=-90,
            max_lat=90,
            max_bounds=True,)
ChoroplethNbrLiaisons(pays,m)
m.get_root().html.add_child(f.Element("<h3 align='center' style='font-size:15px' ><b>Liaisons depuis {}</b></h3>".format(pays)))
f.LayerControl().add_to(m)



m

## Graphiques

Histogramme NbrVols/Distance

In [ ]:
def calculDistLigne(ligne):
    dist = 0
    try : 
        dist = dict_route[frozenset((ligne.Depart,ligne.Arrivee))]['Dist']
    except IndexError:
        #Certains aéroports de la df_vols ne sont pas dans le dict...
        pass

    return dist

In [ ]:
df_volsDist = df_vols.copy()
df_volsDist['Dist'] = df_volsDist.apply(calculDistLigne, axis=1)

In [ ]:
#On ajoute 10km pour corriger un bug d'affichage...
df_volsDist['Dist'] = df_volsDist['Dist'] + 10
fig = px.histogram(df_volsDist, x="Dist",nbins = 60, log_y=True, marginal="box" ,width=800, height=550, range_x=(0,16000),
                    title ="Histogramme des vols en fonction de la distance",
                    labels={"Dist" : "Distance (km)","count" : "Nbr de vols"})
fig.show()

In [ ]:
# Courbe de dissuasion.
# Faire apparaitre une courbe nombre de bins reduite.
# Clustering sur les graphes.
# Trouver des liaisons pas normals vis à vis de la distance (Ancienne colonie, comparer graphe des distances.)
# Export appli grâce aux widgets.
# Découper la fonction de map.
# Integrer widget (Regarder ton code)
# Popup Chloropeth

Distance par compagnie

In [ ]:
def ajoutColTypeDest(ligne, boolCompagnie : bool = False):
    """
        Fonction : Exprime le type de vol en fonction du pays d'origine OU du pays de la compagnie
                    Pays par defaut
        Retour : National, Continental ou International
    """
    if(boolCompagnie):
        dict_tmp = dict_compagnies
        col = ligne.Code_ICAO
    else:
        dict_tmp = dict_aeroports
        col = ligne.Depart

    try :
        if(dict_tmp[col]['Country'] == dict_aeroports[ligne.Arrivee]['Country']):
            TypeDest = 'National'
        elif(dict_pays[dict_tmp[col]['Country']] == dict_pays[dict_aeroports[ligne.Arrivee]['Country']] ):
            TypeDest = 'Continental'
        else:
            TypeDest = 'International'
        return TypeDest
    except:
        pass
    
#Type de vol en fonction des Compagnies et des arrivées
#df_vols['TypeDest'] = df_vols.apply(ajoutColTypeDest, boolCompagnie=True, axis=1)
#df_vols.head(10)

In [ ]:
#Boxplot plotly
def boxplotDistanceParCompagnie(listCompagnie : list):
    #Filtre sur les compagnies
    df = df_volsDist[df_volsDist['Code_ICAO'].isin(listCompagnie)].copy()

    #On ajoute le type de destination
    df['TypeDest'] = df.apply(ajoutColTypeDest, axis=1)

    #On affiche
    fig = px.box(df,x='TypeDest',y='Dist', color=[dict_compagnies[x]['Name'] for x in df['Code_ICAO']],
                labels={"color" : "Compagnies", "TypeDest" : "Type de destination", "Dist" : "Distance (km)"},
                title="Boxplot des distances en fonction du type de destination",
                width=900,
                height=500)
    fig.show()

boxplotDistanceParCompagnie(['DLH','AFR','ACA'])


Bar Plot NbrAeroports/Compagnies

In [ ]:
def supprDoublonEtGroupByTypeDest(df : pd.DataFrame,listDoublon : list, listGroup : list):
    """
        Fonction : elimine les doublons puis groupe la dataframe Df_vols en fonction
                    du type de la destination
                    IMPORTANT : ajouter 'TypeDest' dans listGroup
        Retour : Dataframe
    """
    #Suppression des doublons
    if(listDoublon):
        df_tmp = df.drop_duplicates(subset =listDoublon, keep = 'first')
    else:
        df_tmp = df

    #Groupby puis mise en forme de la df
    if(listGroup):
        df_tmp = df_tmp.groupby(listGroup).size()
        df_tmp = df_tmp.unstack()
        df_tmp['Total'] = df_tmp.sum(axis=1)

    
    return df_tmp.sort_values(by='Total', ascending=False)

In [ ]:
def afficheBarplotXXXParCompagnie(df : pd.DataFrame, XXX : str):
    # On affiche
    fig = px.bar(df, y=[dict_compagnies[x]['Name'] for x in df.index], 
                    x=['National', 'Continental', 'International'], 
                    title="Nombre {} desservi par compagnies".format(XXX),
                    labels={'variable' : "Type de destination","y" : "Compagnies","value" : "Nombre {}".format(XXX)},
                    orientation='h',
                    width=800,
                    height=550)

    # On positionne la légende
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ))
    fig.show()

def barplotAeroportParCompagnie():
    #On crée une copie de df_vols
    df_tmp = df_vols.copy()

    #Ajout de la colonne TypeDest:
    df_tmp['TypeDest'] = df_vols.apply(ajoutColTypeDest, boolCompagnie=True, axis=1)
    df_tmp = supprDoublonEtGroupByTypeDest(df_tmp,['Code_ICAO','Arrivee'],['Code_ICAO','TypeDest'])[0:10]

    afficheBarplotXXXParCompagnie(df_tmp,"d'aéroports")
    

barplotAeroportParCompagnie()

Bar Plot Route/Compagnies

In [ ]:
def barplotRouteParCompagnie(): 
    df_tmp = supprDoublonEtGroupByTypeDest(df_volsTypeDest,['Code_ICAO','Depart','Arrivee'],['Code_ICAO','TypeDest'])[0:10]
    afficheBarplotXXXParCompagnie(df_tmp,"de routes")

#Création d'une nouvelle Dataframe contenant le type de destination
df_volsTypeDest = df_vols.copy()
df_volsTypeDest['TypeDest'] = df_vols.apply(ajoutColTypeDest, axis=1)
barplotRouteParCompagnie()

Pie répartition des types de routes

In [ ]:
fig = px.pie(df_volsTypeDest, names='TypeDest', hole=0.4, title="Pourcentage des différents types de vols")
fig.show()